### Mathematical model gap visualization

In [ ]:
from os import listdir, mkdir
from os.path import isfile, join, exists

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
SAVE_IMAGE = False

# Save image as file
def save_gap_image(fig, filename):
    if not exists("images/gap"):
        mkdir("images/gap")

    fig.write_image("images/gap/" + filename + ".pdf")

In [ ]:
def get_files_from_folder(folder):
    return [join(folder, f) for f in listdir(folder) if isfile(join(folder, f))]

# "./foo/bar/n120m8-01.csv" -> "n120m8-01"
def extract_filename(filenames):
    return [f.split("/")[-1].split(".")[0] for f in filenames]

def get_all_results(files, names):
    results = []
    
    for file in files:
        df = pd.read_csv(
            file, header=0, index_col=None, usecols=names
        )

        results.append(df)
        
    return results

# Calculate makespan at time intervals
def calculate_best_makespan(df, num_windows, window_size):
    first_solution = df[df.Objective < 1e+100].Objective.max()
    makespan = [first_solution]

    for window_idx in range(1, num_windows):
        window = window_size * window_idx
    
        makespan_smaller_than = df[df["Time"] <= window]
        
        current_solution = makespan_smaller_than["Objective"].min()
        
        if current_solution < first_solution:
            makespan.append(current_solution)
        else:
            makespan.append(first_solution)
        
    return makespan

# Calculate makespan at time intervals
def calculate_best_bound(df, num_windows, window_size):
    makespan = [df["Best Bound"].min()]

    for window_idx in range(1, num_windows):
        window = window_size * window_idx
        bound_larger_than = df[df["Time"] <= window]
        makespan.append(bound_larger_than["Best Bound"].max())
        
    return makespan

def get_gap_df(folder, names, num_windows):
    files = get_files_from_folder(folder)

    # Get list of DataFrames with all relevant results
    dfs = get_all_results(files, names)

    # Extract name of instance from path to result file for each instance
    files = extract_filename(files)

    # Calculate maximum time from time column
    max_time = dfs[0]["Time"].max()

    # Define number of windows to divide results into and define window size
    window_size = max_time / num_windows

    # Create list of all increments in window size
    index = [round(window_size * i, 0) for i in range(num_windows)]

    res = []

    # Iterate all DataFrames and calculate best makespan at each time interval
    for df in dfs:
        if names[1] == "Objective":
            values = calculate_best_makespan(df, num_windows, window_size)
        elif names[1] == "Best Bound":
            values = calculate_best_bound(df, num_windows, window_size)

        res.append(values)

    # Concatenate all results into one DataFrame
    df = pd.DataFrame(res, columns=index, index=files).transpose()
    
    return df

def calc_stats(dfs):
    stats = []
    for df in dfs:
        stats.append(df.agg(["mean", "sem"], axis=1))
        
    return stats

def plot_gap(folder, num_windows=100):
    dfs = []
    names = ["Objective", "Best Bound"]
    for name in names:
        dfs.append(get_gap_df(folder, ["Time", name], num_windows))
        
    max_df = pd.concat(dfs).groupby(level=0).max().iloc[0]
        
    for i, df in enumerate(dfs):
        if i == 0:
            df = df.transform(lambda x: round(x / x.max() * 100, 1))        
        elif i == 1:
            df = df.transform(lambda x: round(x / max_df[x.name].max() * 100, 1))
        dfs[i] = df
        
    stats = calc_stats(dfs)
    
    fig = go.Figure()
    for i, df in enumerate(stats):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df["mean"],
                name=names[i],
            )
        )

    fig.update_layout(
            template="simple_white",
            xaxis_title="Time (s)",
            yaxis_title="Percentage of initial solution",
            font_family="Times New Roman",
        )

    fig.update_xaxes(mirror=True)
    fig.update_yaxes(mirror=True)
    
    fig.update_yaxes(rangemode="tozero")
    
    return fig

def plot_avg(folder, num_windows=100):
    dfs = []
    names = ["Objective", "Best Bound"]
    for name in names:
        dfs.append(get_gap_df(folder, ["Time", name], num_windows))        
        
    for i, df in enumerate(dfs):
        df["mean"] = df.mean(axis=1)
        dfs[i] = df
        
    fig = go.Figure()
    for i, df in enumerate(dfs):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df["mean"],
                name=names[i],
            )
        )

    fig.update_layout(
            template="simple_white",
            xaxis_title="Time (s)",
            yaxis_title="Percentage of initial solution",
            font_family="Times New Roman",
        )

    fig.update_xaxes(mirror=True)
    fig.update_yaxes(mirror=True)
    
    fig.update_yaxes(rangemode="tozero")
    
    return fig

In [ ]:
fig = plot_avg("../../coding/hffsp/gaps", 200)

fig.update_layout(
    legend=dict(
        x=0.8, 
        y=0.98
    )
)

fig.show()

if True:
    save_gap_image(fig, "gap_average")